In [1]:
import torch
import torch.nn as nn

In [2]:
in_channels = 3

In [3]:
class DepthWiseNet(nn.Module):
    def __init__(self,inchannel,outchannel):
        super(DepthWiseNet,self).__init__()
        self.depthwise=nn.Sequential(nn.Conv2d(inchannel,outchannel,kernel_size=3,stride=1,padding=0,bias=False))
    def forward(self,x):
        return self.depthwise(x)

## 分离卷积
创建3个inchannel=1，outchannel=1的卷积，分别对3个输入通道的图像进行卷积

In [4]:
nets = []
for i in range(in_channels):
    nets.append(DepthWiseNet(1,1))
nets

[DepthWiseNet(
   (depthwise): Sequential(
     (0): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), bias=False)
   )
 ),
 DepthWiseNet(
   (depthwise): Sequential(
     (0): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), bias=False)
   )
 ),
 DepthWiseNet(
   (depthwise): Sequential(
     (0): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), bias=False)
   )
 )]

In [5]:
x = torch.randn(1,3,5,5)
x

tensor([[[[ 0.9934,  0.5529, -1.5584, -0.3948, -0.8393],
          [ 0.0171, -1.4203,  0.5671,  1.4039,  0.6487],
          [-0.9060, -0.7754, -1.5091,  0.8850,  0.5971],
          [-0.4518,  0.0793,  1.2403,  1.3125, -0.4387],
          [ 0.6843, -0.2675,  0.5599,  1.1151, -0.4898]],

         [[ 0.3306, -0.0047, -0.7463, -0.6212, -0.9994],
          [ 1.0173, -0.2164, -0.6809,  1.5103,  0.0642],
          [ 0.2257, -0.0507,  0.4798, -1.1714, -0.6464],
          [ 0.6946,  0.7477, -1.2230,  0.3893,  1.5128],
          [ 0.1184,  1.1873,  0.3176, -2.0176,  1.1123]],

         [[ 0.9289,  0.6692,  0.2542,  1.3595,  1.3742],
          [-0.7226, -1.0381, -0.9920,  1.5727,  0.2699],
          [-0.5069, -1.4744, -0.1697, -0.9456, -0.2717],
          [ 0.1414, -0.1233, -1.5721,  1.0576, -0.2282],
          [ 2.5426,  0.2357, -1.5056, -0.1848, -0.5909]]]])

## 分离输入图像的三通道

In [6]:
def Splitdata(x):
    data=[]
    for e in x[0]:
        data.append(e.unsqueeze(0))
    return data

In [7]:
x = Splitdata(x)
print(x)
print('len of features map: ', len(x))
print(x[0].shape)

[tensor([[[ 0.9934,  0.5529, -1.5584, -0.3948, -0.8393],
         [ 0.0171, -1.4203,  0.5671,  1.4039,  0.6487],
         [-0.9060, -0.7754, -1.5091,  0.8850,  0.5971],
         [-0.4518,  0.0793,  1.2403,  1.3125, -0.4387],
         [ 0.6843, -0.2675,  0.5599,  1.1151, -0.4898]]]), tensor([[[ 0.3306, -0.0047, -0.7463, -0.6212, -0.9994],
         [ 1.0173, -0.2164, -0.6809,  1.5103,  0.0642],
         [ 0.2257, -0.0507,  0.4798, -1.1714, -0.6464],
         [ 0.6946,  0.7477, -1.2230,  0.3893,  1.5128],
         [ 0.1184,  1.1873,  0.3176, -2.0176,  1.1123]]]), tensor([[[ 0.9289,  0.6692,  0.2542,  1.3595,  1.3742],
         [-0.7226, -1.0381, -0.9920,  1.5727,  0.2699],
         [-0.5069, -1.4744, -0.1697, -0.9456, -0.2717],
         [ 0.1414, -0.1233, -1.5721,  1.0576, -0.2282],
         [ 2.5426,  0.2357, -1.5056, -0.1848, -0.5909]]])]
len of features map:  3
torch.Size([1, 5, 5])


## 逐层进行卷积，最后stack

In [8]:
print(len(x))
for i in range(len(x)):
    x[i]= nets[i](x[i].unsqueeze(0)).squeeze(0)
    x[i]=x[i].squeeze(0)
data = torch.stack(x)
data = data.unsqueeze(0)
print('shape: ', data.shape)
data

3
shape:  torch.Size([1, 3, 3, 3])


tensor([[[[-1.5158,  0.9021,  0.8251],
          [ 0.7186,  1.2603,  0.6302],
          [ 0.4121,  1.7960,  0.0994]],

         [[-0.3979,  0.2852,  0.2325],
          [-0.1007, -0.3794, -0.0547],
          [ 0.1370, -0.2497, -0.2744]],

         [[ 0.4448,  0.6724,  0.2236],
          [-0.3396, -0.1736, -0.0340],
          [-1.0658, -0.1118,  0.1071]]]], grad_fn=<UnsqueezeBackward0>)